In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

Join together UW orgs with the CRA/211 data

In [3]:
df_orgs = pd.read_csv('../joined-data/simplified_matches.csv')

df_uw = pd.read_excel('../cra-data/cre-uw-checking 2.xlsx', sheet_name="UW Orgs")
df_uw = df_uw.rename(columns={'CHARITABLE REGISTRATION NUMBER (BN)': 'CRA_BN_ID'})

print(f'Total UW orgs: {len(df_uw)}')

# df_uw = df_uw[df_uw['CRA_BN_ID'].notna()]
# print(f'Total UW orgs (with CRA BN ID): {len(df_uw)}')

num_uw_matches = len(set(df_uw['CRA_BN_ID']).intersection(set(df_orgs['CRA_BN_ID'])))
print(f'No. UW orgs with 211/CRA data: {num_uw_matches}')

Total UW orgs: 278
No. UW orgs with 211/CRA data: 167


In [4]:
df_uw_cra211 = pd.merge(
    left = df_uw,
    right = df_orgs,
    on = 'CRA_BN_ID',
    how = 'left'
)
df_uw_cra211 = df_uw_cra211.sort_values(by='CRA_Organization_Name')
df_uw_cra211.to_csv('../joined-data/uw_cra211.csv', index=False)

Join tenure data

In [ ]:
# Problem: need to join on multiple keys due to multiple branches have the same CRA ID

In [26]:
df_tenure_by_cra = pd.read_excel('../../2024-work/SPRE_data/2021_CRE_DATA_Oct9.xlsx')
df_tenure_by_cra = df_tenure_by_cra[['CRA_BN', 'Tenure', 'Latitude', 'Longitude']].rename(
    columns={
        'CRA_BN': 'CRA_BN_ID',
        'Latitude': 'Y_Coordinate',
        'Longitude': 'X_Coordinate',
    }
)

In [27]:
df_tenure_by_cra.columns

Index(['CRA_BN_ID', 'Tenure', 'Y_Coordinate', 'X_Coordinate'], dtype='object')

In [29]:
pd.merge(
    left = df_uw_cra211,
    right = df_tenure_by_cra,
    on = ['CRA_BN_ID', 'Y_Coordinate', 'X_Coordinate'],
    how = 'left'
) #.to_csv('../joined-data/uw_cra211_tenure.csv', index=False)

,Funded Agency - CRA Name,CRA_BN_ID,Category,Sub Category,City,Reason,Region,CRA_Organization_Name,211_Organization_Name,211_Location_Name,211_Address,211_Postal_Code,X_Coordinate,Y_Coordinate,Match_Method,Tenure
0,519 Church Street Community Centre,119310761RR0001,200.0,5.0,TORONTO,Categories not included in filter,Toronto,519 CHURCH STREET COMMUNITY CENTRE,519 Community Centre,519 Community Centre - 519 Community Centre,519 Church St,M4Y 2C9,-79.381005,43.666511,Postal Code + Fuzzy Name,NaN
1,519 Church Street Community Centre,119310761RR0001,200.0,5.0,TORONTO,Categories not included in filter,Toronto,519 CHURCH STREET COMMUNITY CENTRE,519 Community Centre,519 Community Centre - Settlement services,519 Church St,M4Y 2C9,-79.381005,43.666511,Postal Code + Fuzzy Name,NaN
2,519 Church Street Community Centre,119310761RR0001,200.0,5.0,TORONTO,Categories not included in filter,Toronto,519 CHURCH STREET COMMUNITY CENTRE,519 Community Centre,"519 Community Centre - Family, Youth and Child...",519 Church St,M4Y 2C9,-79.381005,43.666511,Postal Code + Fuzzy Name,NaN
3,Aboriginal Legal Services Inc.,129565040RR0001,160.0,9.0,TORONTO,In Data,Toronto,ABORIGINAL LEGAL SERVICES INC.,Aboriginal Legal Services,Aboriginal Legal Services - Aboriginal Legal S...,211 Yonge St,M5B 1M4,-79.379566,43.653840,Postal Code + Fuzzy Name,Rent
4,Abrigo Centre,129515284RR0001,1.0,99.0,TORONTO,In Data,Toronto,ABRIGO CENTRE,Abrigo Centre,Abrigo Centre - Abrigo Centre,1645 Dufferin St,M6H 3L9,-79.442615,43.677187,Exact Name,Rent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,Restoration and Empowerment For Social Transit...,811570365RR0001,160.0,19.0,BRAMPTON,Not joined to 211 data,Peel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1924,The Canadian Red Cross Society La Soci�t� Cana...,119219814RR0001,100.0,99.0,OTTAWA,Postal Code not included in filter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1925,Dnaagdawenmag Binnoojiiyag Child and Family Se...,830618088RR0001,160.0,19.0,HIAWATHA,Postal Code not included in filter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926,Routes Connecting Communities Inc.,888551868RR0001,1.0,99.0,SUTTON WEST,Postal Code not included in filter,York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
